# HDS5210-2020 Midterm

In the midterm, you're going to focus on using the programming skills that you've developed so far to build a calculator for the Apache II scoring system for ICU Mortality.  
* https://www.mdcalc.com/apache-ii-score#evidence
* https://reference.medscape.com/calculator/apache-ii-scoring-system

For the midterm, we'll be building a calculator for the Apache II score and then running that against a patient file that's available to you out on the internet.  This will be broken down into three main steps:
1. Create your JSON file to encapsulate all of the calculation rules for Apache II
2. Create functions to calculate the Apache II score using your JSON configuration
3. Create a function to loop over the patients in a file on the internet and calculate Apach II scores for all of them



---

## Part 1: Creating a JSON Rules File

Look at the rules for the Apache II scoring system on the pages above.  The first step in the midterm is to use those rules and create a JSON configuration file as described in the 2019 midterm video.  I've provided a starter file named `apache.json` to get you started.

Inside that file, you'll find placeholders for all of the measures that go into the Apache II scoring model:
* Organ Failure History
* Age
* Temperature
* [pH](https://en.wikipedia.org/wiki/PH)
* Heart rate
* Respiratory rate
* [Sodium](https://www.mayoclinic.org/diseases-conditions/hyponatremia/symptoms-causes/syc-20373711)
* [Potassium](https://www.emedicinehealth.com/hyperkalemia/article_em.htm)
* [Creatinine](https://www.medicalnewstoday.com/articles/322380)
* [Hematocrit](https://labtestsonline.org/tests/hematocrit)
* White Blood Count
* [FiO2](https://www.ausmed.com/cpd/articles/oxygen-flow-rate-and-fio2)
* [PaO2](https://www.verywellhealth.com/partial-pressure-of-oyxgen-pa02-914920)
* [A-a gradient](https://www.ncbi.nlm.nih.gov/books/NBK545153/)


You may need to create a sort of nested set of rules in some cases.  For instance, the rule for Creatinine says to use certain ranges and points in the case of Acute Renal Failure and a different set of points for Chronic Renal Failure.

Similarly, the rule for FiO2 says to use PaO2 to calculate scores if the FiO2 is <50, and to use A-a Gradient if the PaO2 is >50.

When you've created your `apache.json` file, make sure it's in the same directory as this notebook.

### Testing your JSON

The assert() functions below should all run just fine.  If you want to change the names of any of the keys in the JSON I provided you, you may, but you'll also need to update this test code so that it doesn't fail.  Remember, your notebook should be able to run end-to-end before you submit it.

In [1]:
import json

with open('apache.json') as f:
    rules = json.load(f)

assert('Organ Failure History' in rules.keys())
assert('Age' in rules.keys())
assert('Temperature' in rules.keys())
assert('pH' in rules.keys())
assert('Heart Rate' in rules.keys())
assert('Respiratory Rate' in rules.keys())
assert('Sodium' in rules.keys())
assert('Potassium' in rules.keys())
assert('Creatinine' in rules.keys())
assert('Hematocrit' in rules.keys())
assert('White Blood Count' in rules.keys())
assert('FiO2' in rules.keys())
assert("Glasgow" in rules.keys())

---

## Part 2: Functions to evaluate rules

Write a series of functions, enough to satisfy all of the main criteria that we're using to calculate the Apache II score.  That list is the same as the assert statements above.

* Each of your functions should be well documented.
* Each function should have "config_file" as one of it's parameters.
* Each function should return a numerical score value.
* Similar to what we discussed in the review, if you can generalize some rules, do so.  You should **NOT** end up with one function for each input variable.  If you did that, you'd have a lot of repetative code.

The Glasgow Coma Scale is simply a 1-to-1 score translation.  Simply add the Glasgow Coma Scale value.  So, you don't need to write a function for this. [Glasgow Coma Scale](https://www.cdc.gov/masstrauma/resources/gcs.pdf)

**CORRECTION ADDED 2/29** - The Glasgow Coma Scale points should be calculated as `1 - Glasgow Coma Scale` rather than what I just stated above.  My preference would be that you do the calculation correctly, as per MDCalc, and then use the **corrected** scores files to compare against as noted in Part 4.

In [35]:
import json

config_file = "apache.json"

def one(organ, config_file):
    """Lines the input of organ up with the dictionary and adds the points score to acore_a"""
    score_a = 0
    config = json.load(open(config_file))
    one_score = config.get("Organ Failure History")
    for i in one_score:
        if i in organ:
            score_a += one_score.get(i)
        
    return score_a
        
def two(age, config_file):
    """Takes the input age and fines which min and max it falls between and get the points score and adds it to score_b"""
    score_b = 0 
    config = json.load(open(config_file))
    age_score = config.get("Age")
    for i in age_score:
        if age >= i.get("min") and age < i.get("max"):
            score_b = i.get("points")
    return score_b

#a_c = Acute or Chronic Renal Failure, mg_per_ml = Creatinine Level
def three(a_c, mg_per_ml, config_file):
    """Takes the inpute a_c and mg_per_ml and determines if the the input a_c is acute or chronic and then gets the correct scoring system
    to get the correct points score based on mg_per_ml and adds that to score_c"""
    score_c = 0
    config = json.load(open(config_file))
    acute_score = config.get("Creatinine")
    failure = {}
    
    for i in acute_score:
        if a_c == 'Acute Renal Failure':
            failure = acute_score.get("Acute Renal Failure")            
        elif a_c == 'Chronic Renal Failure':
            failure = acute_score.get("Chronic Renal Failure")
    for j in failure:
        if mg_per_ml >= j.get("min") and mg_per_ml < j.get("max"):
            score_c = j.get("points")
    return score_c

#fio2 = FiO2 level, oxygen = PaO2 or a-a gradient
def four(fio2,oxygen,config_file):
    """Gets the right scoring system based off of fio2 level then finds what score to get to add to score_d based off of the oxygen level"""
    config = json.load(open(config_file))
    ox_level = config.get("FiO2")
    a_a = {}
    score_d = 0
    for i in ox_level:
        if fio2 >= i.get("min") and fio2 < i.get("max"):
            a_a = i.get("O2_1 Score") or i.get("O2 Score")
            
    for j in a_a:
        if oxygen >= j.get("min") and oxygen < j.get("max"):
            score_d = j.get("points")
            
    return score_d

def five(ph,config_file):
    """Takes the input ph and fines which min and max it falls between and get the points score and adds it to score_e"""
    score_e = 0
    config = json.load(open(config_file))
    ph_score = config.get("pH")
    for i in ph_score:
        if ph >= i.get("min") and ph < i.get("max"):
            score_e = i.get("points")
    return score_e
def six(heart_rate,config_file):
    """Takes the input heart_rate and fines which min and max it falls between and get the points score and adds it to score_f"""
    score_f =0
    config = json.load(open(config_file))
    hr_score=config.get("Heart Rate")
    for i in hr_score:
        if heart_rate >= i.get("min") and heart_rate < i.get("max"):
            score_f = i.get("points")
    return score_f
#res_rate = repiratory rate
def seven(res_rate,config_file):
    """Takes the input res_rate and fines which min and max it falls between and get the points score and adds it to score_g"""
    config = json.load(open(config_file))
    score_g=0
    res_score = config.get("Respiratory Rate")
    for i in res_score:
        if res_rate >= i.get("min") and res_rate < i.get("max"):
            score_g = i.get("points")
    return score_g
#na = sodium
def eight(na,config_file):
    """Takes the input na and fines which min and max it falls between and get the points score and adds it to score_h"""
    config = json.load(open(config_file))
    score_h=0
    na_score = config.get("Sodium")
    for i in na_score:
        if na >= i.get("min") and na < i.get("max"):
            score_h = i.get("points")
        return score_h
#k=potassium
def nine(k,config_file):
    """Takes the input k and fines which min and max it falls between and get the points score and adds it to score_i"""
    config = json.load(open(config_file))
    score_i = 0
    k_score = config.get("Potassium")
    for i in k_score:
        if k >= i.get("min") and k < i.get("max"):
            score_i = i.get("points")
    return score_i
#crit = hematocrit
def ten(crit,config_file):
    """Takes the input crit and fines which min and max it falls between and get the points score and adds it to score_j"""
    config = json.load(open(config_file))
    score_j = 0
    crit_score = config.get("Hematocrit")
    for i in crit_score:
        if crit >= i.get("min") and crit < i.get("max"):
            score_j = i.get("points")
    return score_j
#white = white blood count
def eleven(white,config_file):
    """Takes the input white and fines which min and max it falls between and get the points score and adds it to score_k"""
    config = json.load(open(config_file))
    score_k = 0
    white_score = config.get("White Blood Count")
    for i in white_score:
        if white >= i.get("min") and white < i.get("max"):
            score_k = i.get("points")
    return score_k
def twelve(temp,config_file):
    """Takes the input temp and fines which min and max it falls between and get the points score and adds it to score_l"""
    config = json.load(open(config_file))
    score_l = 0
    temp_score = config.get("Temperature")
    for i in temp_score:
        if temp >= i.get("min") and temp < i.get("max"):
            score_l = i.get("points")
    return score_l
def thirteen(glasgow,config_file):
    """Takes the input glasgow and finds which value it is equal to between 0 and 15 and then takes 1 minus that alue and adds it
    to score_m"""
    config = json.load(open(config_file))
    score_m = 0
    glasgow_score = config.get("Glasgow")
    for i in glasgow_score:
        if glasgow == glasgow_score[i]:
            score_m = 1-glasgow
    return score_m

### Testing you Functions

Write enough test cases to verify that your functions work for evaulating all of the scoring inputs.  Have at least 3 test cases for each input.

These tests can be written the same as the assertions we've use in previous assignments.  For example, if you a function for `temperature_score` then you write a test case like:

```
assert( temperature_score(37) == 0 )
```

In [36]:
assert(one("Elective",config_file)==2)
assert(five(7.65,config_file)==3)
assert(six(1,config_file)==4)
assert(seven(40,config_file)==3)
assert(eight(1,config_file)==4)
assert(nine(1,config_file)==4)
assert(ten(1,config_file)==4)
assert(eleven(0.5,config_file)==4)
assert(twelve(1,config_file)==4)
assert(three("Acute Renal Failure",1.75,config_file)==4)

---

## Part 3: Put it all together

Create a new function called `apache_score()` that takes all of the necessary inputs and returns the final Apache II score.  Use any variable names that you want.  For clarity and organization, my recommendation is to create them in the same order as they're documented in the website.

1. Organ Failure History
2. Age
3. Temperature
4. pH 
5. Heart rate
6. Respiratory rate
7. Sodium
8. Potassium
9. Creatinine
10. Acute renal failure
11. Hematocrit
12. White Blood Count
13. Glasgow Coma Scale
14. FiO2
15. PaO2
16. A-a gradient


In [37]:
import json

def apache_score(organ,age,temp,ph,heart_rate,res_rate,na,k,mg_per_ml,a_c,crit,white,glasgow,fio2,oxygen,config_file):
    final_score =0
    config = json.load(open(config_file))
    final_score += one(organ,config_file)
    final_score += two(age,config_file)
    final_score += three(a_c,mg_per_ml,config_file)
    final_score += four(fio2,oxygen,config_file)
    final_score += five(ph,config_file)
    final_score += six(heart_rate,config_file)
    final_score += seven(res_rate,config_file)
    final_score += eight(na,config_file)
    final_score += nine(k,config_file)
    final_score += ten(crit,config_file)
    final_score += eleven(white,config_file)
    final_score += twelve(temp,config_file)
    final_score += thirteen(glasgow,config_file)
    
    return final_score 

### Testing your Function

Write a few test cases to make sure that your code functions correctly.  In the last step, you'll have LOTS of test cases run through, but you should do some of your before moving on.

In [38]:
import json
organ = "Elective"
age = 47
a_c = "Acute Renal Failure"
mg_per_ml = 0.7
fio2 = 75
oxygen = 250 
ph =7.65
heart_rate = 30
res_rate = 40  
na = 3
k = 4 #20
crit = 47
white = 2
temp = 31 
glasgow = 10
config_file = "apache.json"
apache_score(organ,age,temp,ph,heart_rate,res_rate,na,k,mg_per_ml,a_c,crit,white,glasgow,fio2,oxygen,config_file)

#My score are off by two, but I could not get looping through all patients correct later in this exam, so I did not have time to go back
#and fix my functions

17

---

## Part 4: Accessing and processing the patient file

Fill out the simple function below to retrieve the patient data as a CSV file from any given URL and return a list of all of the Apache II scores based on the data you find for those patients.
* The patient file will be a CSV
* It will have column headers that match the labels shown above
* The columns will not necessarily appear in the order shown above
* You should output only the Apache II scores, not any other information
* Your output should be a list in the same order as the input rows

In [39]:
import requests
import json
import pandas as pd

url = "https://hds5210-2020.s3.amazonaws.com/TestPatients.csv"
rsp = requests.get(url)
x = pd.read_csv(url)

def fourteen(x,config_file):
    for i in x.iterrows():
        #tried this didn't work -> organ = x[i,"Organ Failure History"]
        #tried this didn't work -> organ = x.iloc[i,1]
        organ = x.get("Organ Failure History")
        age = x.get("Age").any()
        a_c = x.get("Acute Renial Failure")
        mg_per_ml = x.get("Creatinine")
        temp = x.get("Temperature").any()
        ph = x.get("pH").any()
        heart_rate = x.get("Heart Rate").any()
        res_rate = x.get("Respiratory Rate").any()
        na = x.get("Sodium").any()
        k = x.get("Potassium").any()
        crit = x.get("Hematocrit").any()
        white = x.get("White Blood Count").any()
        glasgow = x.get("Glasgow Coma Scale").any()
        fio2 = x.get("FiO2").any()
        for j in [fio2]:
            if j < 50:
                oxygen = x.get("PaO2").any()
            else:
                oxygen = x.get("A-a Gradient").any()
        #I could not find out where to put the "i" into apache_score, so it could loop through each value in apache_score's inputs.
        apache = apache_score(organ,age,temp,ph,heart_rate,res_rate,na,k,mg_per_ml,a_c,crit,white,glasgow,fio2,oxygen,config_file)
        print(apache)

### Testing your Function

The URL for the test data is: https://hds5210-2020.s3.amazonaws.com/TestPatients.csv


You can verify your results by comparing them against this data: https://hds5210-2020.s3.amazonaws.com/Scores.csv

**CORRECTION ADDED 3/29** - If you calculated the Glasgow Coma Scale points as per the actual instructions in MDCalc, then please use this set of corrected scores to compare your results with: https://hds5210-2020.s3.amazonaws.com/Scores_corrected.csv


In [40]:
import json
import pandas as pd
import requests

fourteen(x,config_file)

34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
34
3